# Isentropic Averaging

Isentropic coordinates are quasi-Lagrangian meaning that fluid particales cross isentropic surface only when there is diabatic heating. Computing the residual circulation as a mass-weighted mean on isentropic surfaces provides a reference case to evaluate the quality of a Transformed Eulerian Mean. For a reference of the averaged equations of motion in isentropic coordinates see Chapter 3.9 in Andrews. et al. (1987).

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
import numba

from scipy import stats

from generalized_TEM import TEM, Standard, Generalized, _get_aspect

## Define custom functions for 

- interpolation between log-pressure and isentropic coordinates
- computation of isentropic and TEM variables
- quiver plots in log-pressure coordinates

In [ ]:
@numba.guvectorize(
    "(float64[:], float64[:], float64[:], float64[:])",
    "(n), (n), (m) -> (m)",
    nopython=True,
)
def vectorized_interp(data,x,xi,out):
    '''
        Vecotorized 1-D interpolation
    
        - much faster than looping or np.vectorize
        - does not work with scipy's cublic spline interpolation:
        out[:] = interpolate.interp1d(x,data,kind='cubic',fill_value='extrapolate')(xi)
    '''   
    out[:] = np.interp(xi, x, data)
    
    
def interp_to_isentropic(da,T,tem,lev=np.hstack([np.arange(240,380,1),np.arange(380,560,2)])):
    '''
        Interpolation to levels of constant potential temperature
        
        - vectorized_interp does only accept arguments 'x' that are montonically increasing,
          thus the negation
    '''    
    lev = (-1) * xr.DataArray(lev,dims=('theta',))
    theta = (-1) * tem.potential_temperature(T)
    
    interpolated = xr.apply_ufunc(vectorized_interp,
                                  da,theta,lev,
                                  input_core_dims=[['level'],['level'],['theta']],
                                  output_core_dims=[['theta']],
                                  dask='parallelized',
                                  output_dtypes=[np.double])
    
    interpolated['theta'] = (-1) * lev
    
    return interpolated


def interp_to_height(da,height,tem):
    '''
        Inverse interpolation back to pressure levels
    '''    
    interpolated = xr.apply_ufunc(vectorized_interp,
                                  da,height,tem.lev,
                                  input_core_dims=[['theta'],['theta'],['level']],
                                  output_core_dims=[['level']],
                                  dask='parallelized',
                                  output_dtypes=[np.double])
    interpolated['level'] = tem.lev
    
    return interpolated

In [ ]:
def density(T,tem,lev=np.hstack([np.arange(240,380,1),np.arange(380,560,2)])):
    '''
        Deduce 'density' from the vertical pressure gradient
    '''
    g = 9.81
    ps = 1013.0
    
    plev = ps * np.exp(-tem.lev / tem.H)
    
    pressure = interp_to_isentropic(plev,T,tem,lev=lev)
    
    sigma = - 100 / g * pressure.differentiate('theta') / pressure['theta'].differentiate('theta')
    
    return sigma


def mass_weighted_mean(da,T,tem,lev=np.hstack([np.arange(240,380,1),np.arange(380,560,2)])):
    '''
        Mass-weighted mean as defined by eq. (3.9.5) in Andrews et al. (1987)
    '''
    
    sigma = density(T,tem,lev=lev)
    
    interpolated = interp_to_isentropic(da,T,tem,lev=lev)
    
    result = (interpolated * sigma).mean(tem.dims) / sigma.mean(tem.dims)
    
    return result

def streamfunction_no_mean(tem,v):
    '''
        Compute the mass streamfunction from averaged merdional velocity
    '''
    spacing = tem.lev.values
    spacing = np.insert(np.append(spacing,-spacing[-1]),0,spacing[0])
    spacing = (spacing[:-2]-spacing[2:])/2
        
    spacing = xr.DataArray(spacing,dims=[tem.lev_name],
                           coords={tem.lev_name:tem.lev[tem.lev_name]})
        
    transport = - v * spacing * tem.rho_0 * tem.cos_phi
    transport = transport.cumsum(tem.lev_name)
        
    return transport
    
    
def EP_flux(tem,u,v,omega,eChi):
    '''
        Compute meridional and vertical component of EP-flux from
        velocity data and the eddy-overturning streamfunction eChi
    '''
    yflux, zflux = tem.momentum_flux(u,v,omega)
        
    A, B = tem.angular(u)
    yflux = -yflux + B * eChi
    zflux = -zflux - A * eChi
        
    yflux = tem.a * yflux
    zflux = tem.a * zflux
        
    return yflux, zflux

In [ ]:
def epQuiver(tem,fig,ax,u,v,scale=None,n=22):
    '''
        Produce quiver plot for EP-flux vectors 
        
        Parameters
        ----------
        tem : object
            instance of TEM class
        fig : object
            figure
        ax : object
            axes
        u : xarray.DataArray
            meridional EP-flux component
        v : xarray.DataArray
            vertical EP-flux component
        scale : float, optional
            scale for the lenght of the arrows (default is None)
        n : int, optional
            number of vectors in each dimension (default is 22)
    '''
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # scale vectors following Jucker (2021)
    uhat = 2*np.pi/tem.rho_0.drop('level') * tem.a*tem.cos_phi * u
    vhat = 2*np.pi/tem.rho_0.drop('level') * tem.a*tem.a*tem.cos_phi * v
    
    # for latitude in radians
    #ux = uhat /(xlim[1]-xlim[0])
    # for latitude in degrees
    ux = uhat /(xlim[1]-xlim[0]) / np.pi * 180
    ux = ux * _get_aspect(fig,ax)
    # for level in log_pressure
    vy = vhat / (ylim[1]-ylim[0])
    # for level in pressure
    #vy = vhat / (-tem.H * np.log(ylim[1]/ylim[0]))
    
    # interpolate to regular grid or logarithmic grid
    X = np.linspace(xlim[0],xlim[1],n)
    X = X[1:-1]
    Y = np.linspace(ylim[0],ylim[1],n)
    #Y = np.exp(np.linspace(np.log(ylim[0]),np.log(ylim[1]),n))
    Y = Y[1:-1]
    
    U = ux.interp(latitude=X,level=Y)
    V = vy.interp(latitude=X,level=Y)
    
    q = ax.quiver(X,Y,U,V,scale=scale,color='black')

## Compute streamfunction of residual circulation year by year

This is done in oder to perform a t-test for the significance of the multi-year mean.

In [ ]:
## Eulerian mean and isentropic average

indir = '/work/ERA5_for_TEM/daymean/'
outdir = '/work/isentropic_average/'

files = [f for f in os.listdir(indir) if f.startswith('era5_JJA')]
files.sort()

for f in files:
    
    print(f)
    
    data = xr.open_dataset(indir+f,chunks=dict(time=1))
    
    # define TEM instance for the transformation from pressure to log-pressure coordiantes
    tem = TEM(data['t'],dims=('time','longitude'),H=7000)
    
    # compute the Eulerian mean in log-pressure
    Eulerian = tem.streamfunction(data['v']).compute()
    Eulerian['level'] = tem.lev
    
    # compute isentropic average of meridional wind, log-pressure and pressure
    
    vwind = mass_weighted_mean(data['v'],data['t'],tem).compute()
    height = interp_to_isentropic(tem.lev,data['t'],tem).mean(tem.dims).compute()
    pressure = interp_to_isentropic(data['level'],data['t'],tem).mean(tem.dims).compute()
    
    
    # compute diagnostics in log-pressure
    V = interp_to_height(vwind,height,tem)

    V['level'] = data['level']
    Chi = streamfunction_no_mean(tem,V)
    Chi['level'] = tem.lev
    
    new_height = - tem.H * np.log(pressure / 1013)
    new_isentropic = interp_to_height(lev,new_height,tem)
    
    # store diagnostics to disk
    diagnostics = xr.Dataset(dict(Eulerian=Eulerian,Chi=Chi,theta=new_isentropic))
    
    diagnostics.to_netcdf(outdir+f)


In [ ]:
## standard and generalized form of TEM

indir = '/work/ERA5_for_TEM/daymean/'
outdir = '/work/TEM_year_by_year/'

files = [f for f in os.listdir(indir) if f.startswith('era5_JJA')]
files.sort()

for f in files:
    
    print(f)
    
    data = xr.open_dataset(indir+f,chunks=dict(time=1))
    
    # create TEM instances
    standardTEM = Standard(data['t'],dims=('time','longitude'),H=7000)
    generalizedTEM = Generalized(data['t'],dims=('time','longitude'),H=7000)
    
    level = standardTEM.lev
    
    # compute eddy overturning
    standard = standardTEM.eChi(data['v'],data['w'],data['t']).compute()
    generalized = generalizedTEM.eChi(data['v'],data['w'],data['t'],order=3).compute()
        
    # store diagnostics to disk
    diagnostics = xr.Dataset(dict(standard=standard,generalized=generalized))
    diagnostics['level'] = level
    
    diagnostics.to_netcdf(outdir+f)

## Compare the residual circulation streamfunction

Compute multi-year mean for each version of the TEM and estimate difference to the isentropic average using a t-test. Note that for plotting, the sign of the streamfunction is inverted. 

In [ ]:
## Read diagnostics

years = np.arange(1979,2022)

isentropic = []
tem = []

for y in years:
    
    ds = xr.open_dataset('/work/isentropic_average/era5_newDJF_%d.nc'%y)
    isentropic.append(ds.assign_coords(year=y))
    
    ds = xr.open_dataset('/work/TEM_year_by_year/era5_newDJF_%d.nc'%y)
    tem.append(ds.assign_coords(year=y))
    
isentropic = xr.concat(isentropic,dim='year')
tem = xr.concat(tem,dim='year')


## Plotting

fig, axes = plt.subplots(nrows=3,ncols=2,sharex=True,sharey=True,figsize=(9,12))
axes = axes.flatten()

Chi_levels = np.linspace(-6000,6000,33)
theta_levels = np.arange(260,560,20)


# first plot
Chi = (-1) * isentropic['Chi']
theta = isentropic['theta'].mean('year')

C0 = Chi.mean('year').plot.contourf(ax=axes[0],x='latitude',levels=Chi_levels,
                                                      cmap=cmocean.cm.balance,extend='both',add_colorbar=False)
C = theta.plot.contour(ax=axes[0],x='latitude',levels=theta_levels,
                                   colors='k',add_colorbar=False)
plt.clabel(C)


# second plot
C1 = (Chi.where(Chi['level']>21000)).mean('year').plot.contourf(ax=axes[1],x='latitude',levels=Chi_levels/3,
                                                      cmap=cmocean.cm.balance,extend='both',add_colorbar=False)
C = theta.plot.contour(ax=axes[1],x='latitude',levels=theta_levels,
                                   colors='k',add_colorbar=False)
plt.clabel(C)


# third plot
standard = -1 * tem['standard'] - isentropic['Eulerian']

C0 = standard.mean('year').plot.contourf(ax=axes[2],x='latitude',levels=Chi_levels,
                                                      cmap=cmocean.cm.balance,extend='both',add_colorbar=False)
C = theta.plot.contour(ax=axes[2],x='latitude',levels=theta_levels,
                                   colors='k',add_colorbar=False)
plt.clabel(C)


# fourth plot
difference = standard.mean('year') - Chi.mean('year')

n = len(Chi['year'])
t = difference * (standard.var('year')+Chi.var('year'))**(-0.5) * n**(0.5)
dof = n - 1 + (2*n-2) / (standard.var('year')/Chi.var('year') + Chi.var('year')/standard.var('year'))

sig = (t > stats.t.isf(0.025,dof)) + (t < stats.t.isf(0.975,dof))

C1 = difference.plot.contourf(ax=axes[3],x='latitude',levels=Chi_levels/3,
                                                      cmap=cmocean.cm.balance,extend='both',add_colorbar=False)
sig.astype(np.double).plot.contourf(ax=axes[3],x='latitude',levels=[0,0.5,1],hatches=['//',''],alpha=0,add_colorbar=False)
C = theta.plot.contour(ax=axes[3],x='latitude',levels=theta_levels,
                                   colors='k',add_colorbar=False)
plt.clabel(C)


# fifth plot
generalized = -1 * tem['generalized'] - isentropic['Eulerian']

C0 = generalized.mean('year').plot.contourf(ax=axes[4],x='latitude',levels=Chi_levels,
                                                      cmap=cmocean.cm.balance,extend='both',add_colorbar=False)
C = theta.plot.contour(ax=axes[4],x='latitude',levels=theta_levels,
                                   colors='k',add_colorbar=False)
plt.clabel(C)


# sixth plot
difference = generalized.mean('year') - Chi.mean('year')

n = len(Chi['year'])
t = difference * (generalized.var('year')+Chi.var('year'))**(-0.5) * n**(0.5)
dof = n - 1 + (2*n-2) / (generalized.var('year')/Chi.var('year') + Chi.var('year')/generalized.var('year'))

sig = (t > stats.t.isf(0.025,dof)) + (t < stats.t.isf(0.975,dof))

C1 = difference.plot.contourf(ax=axes[5],x='latitude',levels=Chi_levels/3,
                                                      cmap=cmocean.cm.balance,extend='both',add_colorbar=False)
sig.astype(np.double).plot.contourf(ax=axes[5],x='latitude',levels=[0,0.5,1],hatches=['//',''],alpha=0,add_colorbar=False)
C = theta.plot.contour(ax=axes[5],x='latitude',levels=theta_levels,
                                   colors='k',add_colorbar=False)
plt.clabel(C)


# config axes

for ax in axes:
    ax.set_ylabel(None)
    ax.set_xlabel(None)
    ax.set_xticks([-60,-30,0,30,60])
    ax.set_ylim(0,20000)
    ax.set_yticks([5000,10000,15000,20000])
    ax.set_yticks([2500,7500,12500,17500],minor=True)
    
axes[0].set_ylabel('Height [m]')
axes[2].set_ylabel('Height [m]')
axes[4].set_ylabel('Height [m]')
axes[4].set_xlabel('Latitude [°N]')
axes[5].set_xlabel('Latitude [°N]')


ax = axes[1].twinx()
ax.set_yticks([])
ax.set_ylabel('isentropic',weight='bold',labelpad=10,size=18)

ax = axes[3].twinx()
ax.set_yticks([])
ax.set_ylabel('standard',weight='bold',labelpad=10,size=18)

ax = axes[5].twinx()
ax.set_yticks([])
ax.set_ylabel('generalized',weight='bold',labelpad=10,size=18)

fig.subplots_adjust(0,0,0.9,0.9,0.1,0.1)


# colorbar

cbar = plt.colorbar(C0,ax=[axes[0],axes[2],axes[4],],orientation='horizontal',pad=0.08,fraction=0.04,
                    ticks=[-6000,-4500,-3000,-1500,0,1500,3000,4500,6000],)
cbar.set_label(r'mass streamfunction [kg m$^{-1}$ s$^{-1}$]',weight='bold')
cbar.ax.set_xticklabels([-6000,-4500,-3000,-1500,0,1500,3000,4500,6000],rotation=45)


cbar = plt.colorbar(C1,ax=[axes[1],axes[3],axes[5],],orientation='horizontal',pad=0.08,fraction=0.04,
                    ticks=[-2000,-1500,-1000,-500,0,500,1000,1500,2000],)
cbar.set_label(r'difference [kg m$^{-1}$ s$^{-1}$]',weight='bold')
cbar.ax.set_xticklabels([-2000,-1500,-1000,-500,0,500,1000,1500,2000],rotation=45)


## Compare the EP-flux divergence

Subtract the Eulerian mean overturning for the isentropically averaged residual circulation to estimate eddy-overturning. Then compute the EP-flux from the eddy-angular momentum flux and the eddy-overtuning.

In [ ]:
## compute the EP-flux year-by-year

years = np.arange(1979,2022)

yflux_isentropic = []
zflux_isentropic = []
div_isentropic = []

yflux_standard = []
zflux_standard = []
div_standard = []

yflux_generalized = []
zflux_generalized = []
div_generalized = []

for y in years:
    
    print(y)
    
    isentropic = xr.open_dataset('/work/isentropic_average/era5_JJA_%d.nc'%y)
    tem = xr.open_dataset('/work/TEM_year_by_year/era5_JJA_%d.nc'%y)
    data = xr.open_dataset('/work/ERA5_for_TEM/daymean/era5_JJA_%d.nc'%y,chunks=dict(time=-1))

    tem_tmp = TEM(data['t'],('time','longitude'),H=7000)
    
    # isentropic average
    B_tmp = isentropic['Chi'] - isentropic['Eulerian']
    B_tmp['level'] = data['level']

    yflux_tmp, zflux_tmp = EP_flux(tem_tmp,data['u'],data['v'],data['w'],B_tmp)

    yflux_tmp = yflux_tmp.compute()
    zflux_tmp = zflux_tmp.compute()

    div_tmp = tem_tmp.spher_div(yflux_tmp,zflux_tmp)

    div_tmp['level'] = tem_tmp.lev
    yflux_tmp['level'] = tem_tmp.lev
    zflux_tmp['level'] = tem_tmp.lev
    
    yflux_isentropic.append(yflux_tmp)
    zflux_isentropic.append(zflux_tmp)
    div_isentropic.append(div_tmp)

    
    # standard TEM
    B_tmp = tem['standard']
    B_tmp['level'] = data['level']

    yflux_tmp, zflux_tmp = EP_flux(tem_tmp,data['u'],data['v'],data['w'],B_tmp)

    yflux_tmp = yflux_tmp.compute()
    zflux_tmp = zflux_tmp.compute()

    div_tmp = tem_tmp.spher_div(yflux_tmp,zflux_tmp)

    div_tmp['level'] = tem_tmp.lev
    yflux_tmp['level'] = tem_tmp.lev
    zflux_tmp['level'] = tem_tmp.lev
    
    yflux_standard.append(yflux_tmp)
    zflux_standard.append(zflux_tmp)
    div_standard.append(div_tmp)

    
    # generalized TEM
    B_tmp = tem['generalized']
    B_tmp['level'] = data['level']

    yflux_tmp, zflux_tmp = EP_flux(tem_tmp,data['u'],data['v'],data['w'],B_tmp)

    yflux_tmp = yflux_tmp.compute()
    zflux_tmp = zflux_tmp.compute()

    div_tmp = tem_tmp.spher_div(yflux_tmp,zflux_tmp)

    div_tmp['level'] = tem_tmp.lev
    yflux_tmp['level'] = tem_tmp.lev
    zflux_tmp['level'] = tem_tmp.lev
    
    yflux_generalized.append(yflux_tmp)
    zflux_generalized.append(zflux_tmp)
    div_generalized.append(div_tmp)

In [ ]:
# compute the multiyear avearage
yflux_isentropic = xr.concat(yflux_isentropic,dim='year').mean('year')
zflux_isentropic = xr.concat(zflux_isentropic,dim='year').mean('year')
div_isentropic = xr.concat(div_isentropic,dim='year').mean('year')

yflux_standard = xr.concat(yflux_standard,dim='year').mean('year')
zflux_standard = xr.concat(zflux_standard,dim='year').mean('year')
div_standard = xr.concat(div_standard,dim='year').mean('year')

yflux_generalized = xr.concat(yflux_generalized,dim='year').mean('year')
zflux_generalized = xr.concat(zflux_generalized,dim='year').mean('year')
div_generalized = xr.concat(div_generalized,dim='year').mean('year')

In [ ]:
## Plotting
fig, axes = plt.subplots(nrows=1,ncols=3,sharex=True,sharey=True,figsize=(9,6))
axes = axes.flatten()

C = div_isentropic.plot.contourf(ax=axes[0],x='latitude',levels=np.linspace(-600,600,33),
                  extend='both',cmap=cmocean.cm.balance,add_colorbar=False)

C = div_standard.plot.contourf(ax=axes[1],x='latitude',levels=np.linspace(-600,600,33),
                  extend='both',cmap=cmocean.cm.balance,add_colorbar=False)

C = div_generalized.plot.contourf(ax=axes[2],x='latitude',levels=np.linspace(-600,600,33),
                  extend='both',cmap=cmocean.cm.balance,add_colorbar=False)


# config axes
for ax in axes:
    ax.set_ylabel(None)
    ax.set_xlabel(None)
    ax.set_xticks([-60,-30,0,30,60])
    ax.set_ylim(0,20000)
    ax.set_yticks([5000,10000,15000,20000])
    ax.set_yticks([2500,7500,12500,17500],minor=True)
    
axes[0].set_ylabel('Height [m]')
axes[0].set_xlabel('Latitude [°N]')
axes[1].set_xlabel('Latitude [°N]')
axes[2].set_xlabel('Latitude [°N]')

axes[0].set_title('isentropic',weight='bold',size=18)
axes[1].set_title('standard',weight='bold',size=18)
axes[2].set_title('generalized',weight='bold',size=18)

fig.subplots_adjust(0,0,0.9,0.9,0.1,0.1)


# colorbar 
cbar = plt.colorbar(C,ax=axes,orientation='horizontal',pad=0.14,
                    ticks=[-600,-450,-300,-150,0,150,300,450,600],)
cbar.set_label(r'EP-flux divergence [kg m$^{-1}$ s$^{-2}$]',weight='bold')


# Quiver plot
epQuiver(tem_tmp,fig,axes[0],yflux_isentropic,zflux_isentropic)
epQuiver(tem_tmp,fig,axes[1],yflux_standard,zflux_standard)
epQuiver(tem_tmp,fig,axes[2],yflux_generalized,zflux_generalized)
